In [28]:
NOTEBOOK_NAME = "e008_make_train"

In [29]:
import os
from glob import glob

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import japanize_matplotlib

from contextlib import contextmanager
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
import shutil
from tqdm.auto import tqdm

%matplotlib inline


# ref: Kaggleコード遺産 https://qiita.com/kaggle_grandmaster-arai-san/items/d59b2fb7142ec7e270a5 
class Timer:
    def __init__(self, logger=None, format_str="{:.3f}[s]", prefix=None, suffix=None, sep=" "):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)


def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
# 再現性確保!
seed_everything(33)

In [30]:
INPUT_DIR = "../data"
OUTPUT_DIR = f"../saved_data/{NOTEBOOK_NAME}"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [31]:
# 学習用のログデータと正解ラベル
train_log = pd.read_csv(os.path.join(INPUT_DIR, "train_log.csv"))
train_label = pd.read_csv(os.path.join(INPUT_DIR, "train_label.csv"))

# 宿のデータ
yado = pd.read_csv(os.path.join(INPUT_DIR, "yado.csv"))

# テスト期間のログデータ
test_log = pd.read_csv(os.path.join(INPUT_DIR, "test_log.csv"))

sample_submission = pd.read_csv(os.path.join(INPUT_DIR, "sample_submission.csv"))

In [32]:
whole_log_df = pd.concat([train_log, test_log], ignore_index=True)

In [33]:
def create_session_yad_df(input_df: pd.DataFrame):
    """input_df には train_log / test_log のいずれかが来ることを想定している"""
    N = 10
    sessions = input_df["session_id"].unique()
    out_df = pd.DataFrame(
        {
            "session_id": [x for s in sessions for x in [s] * N],
            "yad_no": yado["yad_no"].sample(n=N * len(sessions), replace=True).values,
        }
    )

    # ランダムに付け加えたもの以外・同一ログに出現する宿を候補にいれる
    out_df = pd.concat([out_df, input_df[["session_id", "yad_no"]]], ignore_index=True)

    # 重複は意味がないので消します。
    out_df = out_df.drop_duplicates()

    # 見た目を揃えるために session / yad の順番でソートをします
    out_df = out_df.sort_values(["session_id", "yad_no"]).reset_index(drop=True)

    return out_df

In [34]:
train_session_yad = create_session_yad_df(input_df=train_log)

# 予測の際には session ごとに yado に対しての予約確率を出さなくてはなりませんから、同じように session - yado の組を作ります。
# ただし学習時と同じような組で良いか? は議論が必要かもしれません. (ここに現れない宿は予測対象に絶対入らないため)
test_session_yad = create_session_yad_df(input_df=test_log)

In [35]:
# 正解ラベル train_label_df の組み合わせを付与
_df = pd.concat([train_label, train_session_yad], ignore_index=True)

# 重複を削除して
_df = _df.drop_duplicates().reset_index(drop=True)

# 正解ラベルに含まれているレコードの index を配列で取得して
target_index = pd.merge(
    _df.reset_index(), train_label, on=["session_id", "yad_no"], how="inner"
)["index"].values

# 正解Indexに含まれている場合 1 / そうでないと 0 のラベルを作成
_df["reserve"] = _df.index.isin(target_index).astype(int)

# 見た目を揃えるために session / yad でソートしておく
_df = _df.sort_values(["session_id", "yad_no"]).reset_index(drop=True)

_df.head(21)

train_session_yad = _df.copy()

In [36]:
# testデータへの後処理
def remove_last_yad_id(session_yad_df):
    # セッション中一番最後の宿の組を作成
    last_yad_df = whole_log_df.groupby("session_id").tail(1)[["session_id", "yad_no"]]

    # 最後であることがわかるようにラベル is_last を付与
    last_yad_df["is_last"] = 1

    # 引数の session - yad の組み合わせとマージして
    merged = session_yad_df.merge(last_yad_df, on=["session_id", "yad_no"], how="left")

    # is_last **ではない** (i.e. is_last is null) データのみに絞る
    idx_use = merged["is_last"].isnull()
    out_df = session_yad_df[idx_use].reset_index(drop=True)

    return out_df


test_session_yad = remove_last_yad_id(test_session_yad)

In [37]:
train_session_yad.to_pickle(f"{OUTPUT_DIR}/{NOTEBOOK_NAME}_merged_train.pkl")
test_session_yad.to_pickle(f"{OUTPUT_DIR}/{NOTEBOOK_NAME}_merged_test.pkl")